# 라이브러리 로드 및 기본 세팅

In [1]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.3 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from catboost import CatBoostClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, precision_recall_curve

# 데이콘 점수

In [5]:
# 자체 점수 : 0.709086
# target 1 개수 : 45274
# 제출 점수(제출 날짜) : 0.6746597821(2022-01-22 11:58:26)

# 설정

In [6]:
# cat
# 총 제거된 특성(12개)
# min_max_scaler
# one_hot_max_size = 2
# random_state : 5(range(10, 60, 10))
# iterations = 1000
# early_stopping_rounds = 50
# 순서형 특성 설정(4개)
# probability_threshold 설정

In [7]:
# 순서형 특성 (4개)
# 'person_attribute_a_a', 'person_attribute_b', 'person_prefer_e', 'contents_attribute_e'

In [8]:
# 기본 제거 특성(5개)
# 'id',
# 'contents_open_dt',
# 'contents_rn',
# 'person_prefer_f',
# 'person_prefer_g',

In [9]:
# target별 특성 분포 유사한 특성(2개)
# 'h_m_match_yn',
# 'h_s_match_yn',

In [10]:
# 특성 중요도 낮은 특성(5개)
# 'person_prefer_h_3_m',
# 'person_prefer_h_3',
# 'contents_attribute_i',
# 'person_prefer_h_3_l',
# 'person_prefer_h_2_m'

# 데이터 불러오기

In [11]:
# train = pd.read_csv('../data/df_train.csv')
# test = pd.read_csv('../data/df_test.csv')
train = pd.read_csv('/content/drive/MyDrive/GitHub/Dacon/TP2(잡케어)/data/df_train.csv')
test = pd.read_csv('/content/drive/MyDrive/GitHub/Dacon/TP2(잡케어)/data/df_test.csv')

# 데이터 확인

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 63 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   id                      501951 non-null  int64 
 1   d_l_match_yn            501951 non-null  bool  
 2   d_m_match_yn            501951 non-null  bool  
 3   d_s_match_yn            501951 non-null  bool  
 4   h_l_match_yn            501951 non-null  bool  
 5   h_m_match_yn            501951 non-null  bool  
 6   h_s_match_yn            501951 non-null  bool  
 7   person_attribute_a      501951 non-null  int64 
 8   person_attribute_a_1    501951 non-null  int64 
 9   person_attribute_b      501951 non-null  int64 
 10  person_prefer_c         501951 non-null  int64 
 11  person_prefer_d_1       501951 non-null  int64 
 12  person_prefer_d_2       501951 non-null  int64 
 13  person_prefer_d_3       501951 non-null  int64 
 14  person_prefer_e         501951 non-n

In [13]:
train.head()

,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,target,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_m,person_prefer_h_2_l,person_prefer_h_3_m,person_prefer_h_3_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,0,True,True,True,False,False,False,1,4,3,5,275,370,369,8,1,1,4,95,59,3,3,10,2,1,2,1608,275,1,4,139,618822,354805,2020-01-17 12:09:36,1,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,316,3,398,94,368,58,422,94,1607,1606,1605,2016
1,1,False,False,False,True,True,False,1,3,4,1,114,181,175,4,1,1,131,101,96,1,3,5,1,1,2,1608,275,1,4,133,571659,346213,2020-06-18 17:48:52,0,114,109,56,1,175,152,56,1,175,152,56,1,275,274,274,216,417,94,400,94,399,94,417,94,1607,1606,1605,2016
2,2,False,False,False,True,False,False,2,0,3,5,464,175,452,3,1,1,54,263,56,3,1,10,2,1,1,1600,94,1,4,53,399816,206408,2020-07-08 20:00:10,0,464,463,450,377,175,152,56,1,452,451,450,377,92,91,56,1,364,48,528,250,366,48,363,48,1599,1595,1572,2016
3,3,False,False,False,True,False,False,2,0,2,5,703,705,704,3,1,1,72,227,2,1,3,5,1,1,2,1608,275,5,3,74,827967,572323,2020-01-13 18:09:34,0,703,703,690,618,703,703,690,618,703,703,690,618,275,274,274,216,379,71,495,226,315,1,381,71,1607,1606,1605,2016
4,4,True,True,True,False,False,False,1,3,4,5,275,370,369,4,1,1,214,210,209,1,1,10,2,1,2,1608,275,1,4,74,831614,573899,2020-03-09 20:39:22,0,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,483,208,481,208,480,208,381,71,1607,1606,1605,2016


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 62 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      46404 non-null  int64 
 1   d_l_match_yn            46404 non-null  bool  
 2   d_m_match_yn            46404 non-null  bool  
 3   d_s_match_yn            46404 non-null  bool  
 4   h_l_match_yn            46404 non-null  bool  
 5   h_m_match_yn            46404 non-null  bool  
 6   h_s_match_yn            46404 non-null  bool  
 7   person_attribute_a      46404 non-null  int64 
 8   person_attribute_a_1    46404 non-null  int64 
 9   person_attribute_b      46404 non-null  int64 
 10  person_prefer_c         46404 non-null  int64 
 11  person_prefer_d_1       46404 non-null  int64 
 12  person_prefer_d_2       46404 non-null  int64 
 13  person_prefer_d_3       46404 non-null  int64 
 14  person_prefer_e         46404 non-null  int64 
 15  pe

In [15]:
test.head()

,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_m,person_prefer_h_2_l,person_prefer_h_3_m,person_prefer_h_3_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,0,True,False,False,True,True,True,1,1,2,1,857,851,1227,4,1,1,263,56,49,1,3,10,2,1,2,1147,839,1,5,263,393790,236865,2020-12-01 02:24:18,854,853,824,744,847,847,824,744,1227,1227,1227,926,836,831,824,744,528,250,366,48,359,48,528,250,1146,1128,1021,2010
1,1,False,False,False,True,False,False,2,0,2,2,683,1086,662,2,1,1,258,263,49,1,3,5,1,1,2,1611,278,1,4,263,394058,236572,2020-12-17 05:42:53,683,677,659,618,1086,1086,1053,926,662,660,659,618,276,274,274,216,523,250,528,250,359,48,528,250,1610,1606,1605,2016
2,2,True,False,False,True,True,True,2,3,2,1,514,790,1233,0,1,1,177,170,171,3,3,3,1,1,2,1817,490,3,4,177,1002061,704612,2020-12-10 23:33:41,514,509,482,482,784,783,745,744,1227,1227,1227,926,490,490,482,482,453,169,451,169,452,169,453,169,1812,1811,1810,2020
3,3,True,False,False,True,True,True,1,2,2,5,114,181,175,4,1,1,177,170,171,3,3,5,1,1,2,101,150,5,3,177,1000813,704652,2020-12-03 19:44:55,114,109,56,1,175,152,56,1,175,152,56,1,145,123,56,1,453,169,451,169,452,169,453,169,101,100,99,2006
4,4,True,False,False,True,False,False,1,6,4,5,1082,1078,1056,5,1,1,178,177,4,3,1,5,1,1,1,985,1097,1,4,177,111146,704413,2020-12-11 21:24:34,1078,1078,1053,926,1078,1078,1053,926,1056,1054,1053,926,1097,1094,1093,926,454,169,453,169,316,3,453,169,984,980,954,2009


# 필요 데이터 추출

## 훈련에 의미가 적은 열 제거

In [16]:
# 제거된 특성(12개)
# 'id',
# 'contents_open_dt',
# 'contents_rn',
# 'person_prefer_f',
# 'person_prefer_g',
# 'h_m_match_yn',
# 'h_s_match_yn',
# 'person_prefer_h_3_m',
# 'person_prefer_h_3',
# 'contents_attribute_i',
# 'person_prefer_h_3_l',
# 'person_prefer_h_2_m'

In [17]:
# 삭제할 특성
pop_col = ['id','contents_open_dt','contents_rn','person_prefer_f','person_prefer_g',
           'h_m_match_yn','h_s_match_yn',
           'person_prefer_h_3_m','person_prefer_h_3',
           'contents_attribute_i','person_prefer_h_3_l',
           'person_prefer_h_2_m']
len(pop_col)

12

In [18]:
train = train.drop(pop_col, axis=1) 
test = test.drop(pop_col, axis=1)

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 51 columns):
 #   Column                  Non-Null Count   Dtype
---  ------                  --------------   -----
 0   d_l_match_yn            501951 non-null  bool 
 1   d_m_match_yn            501951 non-null  bool 
 2   d_s_match_yn            501951 non-null  bool 
 3   h_l_match_yn            501951 non-null  bool 
 4   person_attribute_a      501951 non-null  int64
 5   person_attribute_a_1    501951 non-null  int64
 6   person_attribute_b      501951 non-null  int64
 7   person_prefer_c         501951 non-null  int64
 8   person_prefer_d_1       501951 non-null  int64
 9   person_prefer_d_2       501951 non-null  int64
 10  person_prefer_d_3       501951 non-null  int64
 11  person_prefer_e         501951 non-null  int64
 12  person_prefer_h_1       501951 non-null  int64
 13  person_prefer_h_2       501951 non-null  int64
 14  contents_attribute_a    501951 non-null  int64
 15  

In [20]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 50 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   d_l_match_yn            46404 non-null  bool 
 1   d_m_match_yn            46404 non-null  bool 
 2   d_s_match_yn            46404 non-null  bool 
 3   h_l_match_yn            46404 non-null  bool 
 4   person_attribute_a      46404 non-null  int64
 5   person_attribute_a_1    46404 non-null  int64
 6   person_attribute_b      46404 non-null  int64
 7   person_prefer_c         46404 non-null  int64
 8   person_prefer_d_1       46404 non-null  int64
 9   person_prefer_d_2       46404 non-null  int64
 10  person_prefer_d_3       46404 non-null  int64
 11  person_prefer_e         46404 non-null  int64
 12  person_prefer_h_1       46404 non-null  int64
 13  person_prefer_h_2       46404 non-null  int64
 14  contents_attribute_a    46404 non-null  int64
 15  contents_attribute_

## 카테고리형 특성 원핫인코딩시 컬럼 명 겹치는 문제 해결위해 특성명 변경

In [21]:
# 카테고리형 특성 원핫인코딩시 컬럼 명 겹치는 문제 해결위해 특성명 변경
train = train.rename(columns={'person_attribute_a_1':'person_attribute_a_a'})
train = train.rename(columns={'contents_attribute_j_1':'contents_attribute_j_a'})
train.head()

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,person_attribute_a,person_attribute_a_a,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,contents_attribute_a,contents_attribute_j_a,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,target,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,True,True,True,False,1,4,3,5,275,370,369,8,4,95,3,10,2,1,2,1608,275,1,4,139,618822,1,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,316,3,94,422,94,1607,1606,1605,2016
1,False,False,False,True,1,3,4,1,114,181,175,4,131,101,3,5,1,1,2,1608,275,1,4,133,571659,0,114,109,56,1,175,152,56,1,175,152,56,1,275,274,274,216,417,94,94,417,94,1607,1606,1605,2016
2,False,False,False,True,2,0,3,5,464,175,452,3,54,263,1,10,2,1,1,1600,94,1,4,53,399816,0,464,463,450,377,175,152,56,1,452,451,450,377,92,91,56,1,364,48,250,363,48,1599,1595,1572,2016
3,False,False,False,True,2,0,2,5,703,705,704,3,72,227,3,5,1,1,2,1608,275,5,3,74,827967,0,703,703,690,618,703,703,690,618,703,703,690,618,275,274,274,216,379,71,226,381,71,1607,1606,1605,2016
4,True,True,True,False,1,3,4,5,275,370,369,4,214,210,1,10,2,1,2,1608,275,1,4,74,831614,0,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,483,208,208,381,71,1607,1606,1605,2016


In [22]:
# 카테고리형 특성 원핫인코딩시 컬럼 명 겹치는 문제 해결위해 특성명 변경
test = test.rename(columns={'person_attribute_a_1':'person_attribute_a_a'})
test = test.rename(columns={'contents_attribute_j_1':'contents_attribute_j_a'})
test.head()

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,person_attribute_a,person_attribute_a_a,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,contents_attribute_a,contents_attribute_j_a,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,True,False,False,True,1,1,2,1,857,851,1227,4,263,56,3,10,2,1,2,1147,839,1,5,263,393790,854,853,824,744,847,847,824,744,1227,1227,1227,926,836,831,824,744,528,250,48,528,250,1146,1128,1021,2010
1,False,False,False,True,2,0,2,2,683,1086,662,2,258,263,3,5,1,1,2,1611,278,1,4,263,394058,683,677,659,618,1086,1086,1053,926,662,660,659,618,276,274,274,216,523,250,250,528,250,1610,1606,1605,2016
2,True,False,False,True,2,3,2,1,514,790,1233,0,177,170,3,3,1,1,2,1817,490,3,4,177,1002061,514,509,482,482,784,783,745,744,1227,1227,1227,926,490,490,482,482,453,169,169,453,169,1812,1811,1810,2020
3,True,False,False,True,1,2,2,5,114,181,175,4,177,170,3,5,1,1,2,101,150,5,3,177,1000813,114,109,56,1,175,152,56,1,175,152,56,1,145,123,56,1,453,169,169,453,169,101,100,99,2006
4,True,False,False,True,1,6,4,5,1082,1078,1056,5,178,177,1,5,1,1,1,985,1097,1,4,177,111146,1078,1078,1053,926,1078,1078,1053,926,1056,1054,1053,926,1097,1094,1093,926,454,169,169,453,169,984,980,954,2009


## 카테고리형 특성으로 지정할 특성

In [23]:
# 원본 데이터에서 순서형 특성
# 'person_attribute_a_a', 'person_attribute_b', 'person_prefer_e', 'contents_attribute_e'

In [24]:
# 순서형 특성 수치형으로 지정 : ⭕
# 'person_attribute_a_a', 'person_attribute_b', 'person_prefer_e', 'contents_attribute_e'

In [25]:
cat_f = train.columns.tolist()
numeric_cat = ['person_attribute_a_a', 'person_attribute_b', 'person_prefer_e', 'contents_attribute_e']
for col in numeric_cat:
  cat_f.remove(col)
cat_f.remove('target')

In [26]:
# 51 - 5 = 46개
len(cat_f)

46

# train, test 셋 준비

In [27]:
train.shape

(501951, 51)

In [28]:
X = train.drop(['target'], axis=1)
X.shape

(501951, 50)

In [29]:
y = train[['target']]
y.shape

(501951, 1)

## MinMaxScaler

In [30]:
mmscaler = MinMaxScaler()
mms_X = mmscaler.fit_transform(X)

In [37]:
mms_X.shape

(501951, 50)

In [33]:
mms_X[:,13]

array([0.29903537, 0.31832797, 0.8392283 , ..., 0.40192926, 0.1511254 ,
       0.09324759])

In [39]:
mms_X_df = pd.DataFrame(mms_X, columns=X.columns)
mms_X_df.head()

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,person_attribute_a,person_attribute_a_a,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,contents_attribute_a,contents_attribute_j_a,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,1.0,1.0,1.0,0.0,0.0,0.571429,0.6,1.0,0.216108,0.291866,0.291069,0.727273,0.006431,0.299035,1.0,1.000000,1.0,0.0,1.0,0.798708,0.216108,0.0,0.363636,0.437908,0.589911,0.216108,0.215936,0.216561,0.171042,0.291069,0.290837,0.234873,0.171042,0.291069,0.290837,0.234873,0.171042,0.216108,0.215936,0.216561,0.171042,0.003937,0.006431,0.299035,0.418327,0.298361,0.798211,0.797714,0.797217,0.6
1,0.0,0.0,0.0,1.0,0.0,0.428571,0.8,0.0,0.087719,0.141148,0.136364,0.363636,0.414791,0.318328,1.0,0.444444,0.0,0.0,1.0,0.798708,0.216108,0.0,0.363636,0.418301,0.544951,0.087719,0.084462,0.042994,0.000000,0.136364,0.118725,0.042994,0.000000,0.136364,0.118725,0.042994,0.000000,0.216108,0.215936,0.216561,0.171042,0.401575,0.299035,0.299035,0.398406,0.298361,0.798211,0.797714,0.797217,0.6
2,0.0,0.0,0.0,1.0,1.0,0.000000,0.6,1.0,0.366826,0.136364,0.357257,0.272727,0.167203,0.839228,0.0,1.000000,1.0,0.0,0.0,0.794732,0.071770,0.0,0.363636,0.156863,0.381134,0.366826,0.366534,0.356688,0.299125,0.136364,0.118725,0.042994,0.000000,0.357257,0.356972,0.356688,0.299125,0.070175,0.070120,0.042994,0.000000,0.192913,0.151125,0.800643,0.183267,0.147541,0.794235,0.792247,0.780815,0.6
3,0.0,0.0,0.0,1.0,1.0,0.000000,0.4,1.0,0.557416,0.559011,0.558214,0.272727,0.225080,0.723473,1.0,0.444444,0.0,0.0,1.0,0.798708,0.216108,1.0,0.272727,0.225490,0.789287,0.557416,0.557769,0.547771,0.490851,0.557416,0.557769,0.547771,0.490851,0.557416,0.557769,0.547771,0.490851,0.216108,0.215936,0.216561,0.171042,0.251969,0.225080,0.723473,0.254980,0.222951,0.798211,0.797714,0.797217,0.6
4,1.0,1.0,1.0,0.0,0.0,0.428571,0.8,1.0,0.216108,0.291866,0.291069,0.363636,0.681672,0.668810,0.0,1.000000,1.0,0.0,1.0,0.798708,0.216108,0.0,0.363636,0.225490,0.792763,0.216108,0.215936,0.216561,0.171042,0.291069,0.290837,0.234873,0.171042,0.291069,0.290837,0.234873,0.171042,0.216108,0.215936,0.216561,0.171042,0.661417,0.665595,0.665595,0.254980,0.222951,0.798211,0.797714,0.797217,0.6


In [43]:
mms_X_df = mms_X_df.astype(str)

In [44]:
mms_X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 50 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   d_l_match_yn            501951 non-null  object
 1   d_m_match_yn            501951 non-null  object
 2   d_s_match_yn            501951 non-null  object
 3   h_l_match_yn            501951 non-null  object
 4   person_attribute_a      501951 non-null  object
 5   person_attribute_a_a    501951 non-null  object
 6   person_attribute_b      501951 non-null  object
 7   person_prefer_c         501951 non-null  object
 8   person_prefer_d_1       501951 non-null  object
 9   person_prefer_d_2       501951 non-null  object
 10  person_prefer_d_3       501951 non-null  object
 11  person_prefer_e         501951 non-null  object
 12  person_prefer_h_1       501951 non-null  object
 13  person_prefer_h_2       501951 non-null  object
 14  contents_attribute_a    501951 non-n

In [45]:
mms_X_df.head()

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,person_attribute_a,person_attribute_a_a,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,contents_attribute_a,contents_attribute_j_a,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,1.0,1.0,1.0,0.0,0.0,0.5714285714285714,0.6000000000000001,1.0,0.2161084529505582,0.291866028708134,0.29106858054226475,0.7272727272727273,0.006430868167202572,0.2990353697749196,1.0,1.0,1.0,0.0,1.0,0.7987077534791251,0.2161084529505582,0.0,0.36363636363636365,0.43790849673202614,0.5899105622698995,0.2161084529505582,0.21593625498007968,0.21656050955414013,0.17104216388225937,0.29106858054226475,0.2908366533864542,0.23487261146496816,0.17104216388225937,0.29106858054226475,0.2908366533864542,0.23487261146496816,0.17104216388225937,0.2161084529505582,0.21593625498007968,0.21656050955414013,0.17104216388225937,0.003937007874015741,0.006430868167202573,0.29903536977491957,0.4183266932270917,0.2983606557377049,0.798210735586481,0.7977137176938369,0.7972166998011928,0.6000000000000085
1,0.0,0.0,0.0,1.0,0.0,0.42857142857142855,0.8,0.0,0.08771929824561404,0.14114832535885166,0.13636363636363635,0.36363636363636365,0.41479099678456594,0.3183279742765273,1.0,0.4444444444444445,0.0,0.0,1.0,0.7987077534791251,0.2161084529505582,0.0,0.36363636363636365,0.41830065359477125,0.5449505146816296,0.08771929824561404,0.08446215139442231,0.04299363057324841,0.0,0.13636363636363635,0.11872509960159364,0.04299363057324841,0.0,0.13636363636363635,0.11872509960159364,0.04299363057324841,0.0,0.2161084529505582,0.21593625498007968,0.21656050955414013,0.17104216388225937,0.40157480314960625,0.29903536977491957,0.29903536977491957,0.3984063745019921,0.2983606557377049,0.798210735586481,0.7977137176938369,0.7972166998011928,0.6000000000000085
2,0.0,0.0,0.0,1.0,1.0,0.0,0.6000000000000001,1.0,0.36682615629984056,0.13636363636363635,0.3572567783094099,0.2727272727272727,0.16720257234726688,0.8392282958199356,0.0,1.0,1.0,0.0,0.0,0.7947316103379721,0.07177033492822968,0.0,0.36363636363636365,0.1568627450980392,0.3811341871004521,0.36682615629984056,0.3665338645418327,0.356687898089172,0.29912490055688146,0.13636363636363635,0.11872509960159364,0.04299363057324841,0.0,0.3572567783094099,0.3569721115537849,0.356687898089172,0.29912490055688146,0.07017543859649124,0.0701195219123506,0.04299363057324841,0.0,0.19291338582677153,0.15112540192926047,0.8006430868167203,0.1832669322709164,0.14754098360655737,0.794234592445328,0.7922465208747514,0.7808151093439363,0.6000000000000085
3,0.0,0.0,0.0,1.0,1.0,0.0,0.4,1.0,0.5574162679425838,0.5590111642743222,0.5582137161084529,0.2727272727272727,0.22508038585209003,0.7234726688102894,1.0,0.4444444444444445,0.0,0.0,1.0,0.7987077534791251,0.2161084529505582,1.0,0.2727272727272727,0.22549019607843138,0.7892865382012169,0.5574162679425838,0.5577689243027889,0.5477707006369427,0.4908512330946699,0.5574162679425838,0.5577689243027889,0.5477707006369427,0.4908512330946699,0.5574162679425838,0.5577689243027889,0.5477707006369427,0.4908512330946699,0.2161084529505582,0.21593625498007968,0.21656050955414013,0.17104216388225937,0.25196850393700787,0.22508038585209003,0.7234726688102894,0.2549800796812749,0.22295081967213115,0.798210735586481,0.7977137176938369,0.79721

In [34]:
mms_test = mmscaler.fit_transform(test)

In [41]:
mms_test_df = pd.DataFrame(mms_test, columns=test.columns)
mms_test_df.head()

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,person_attribute_a,person_attribute_a_a,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,contents_attribute_a,contents_attribute_j_a,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,1.0,0.0,0.0,1.0,0.0,0.142857,0.4,0.00,0.680511,0.676518,0.975279,0.363636,0.839228,0.173633,1.0,1.000000,1.0,0.0,1.0,0.573861,0.666933,0.0,0.454545,0.843137,0.375045,0.679744,0.679456,0.666667,0.602107,0.6744,0.674660,0.666667,0.602107,0.975279,0.975299,0.975318,0.735879,0.6656,0.661871,0.666667,0.602107,0.838583,0.800643,0.151125,0.840637,0.809836,0.573647,0.564912,0.512305,0.315789
1,0.0,0.0,0.0,1.0,1.0,0.000000,0.4,0.25,0.541534,0.864217,0.524721,0.181818,0.823151,0.839228,1.0,0.444444,0.0,0.0,1.0,0.806209,0.218850,0.0,0.363636,0.843137,0.375301,0.542834,0.538769,0.532847,0.500000,0.8656,0.865707,0.852393,0.749595,0.524721,0.523506,0.523089,0.490851,0.2176,0.216627,0.220600,0.174230,0.818898,0.800643,0.800643,0.840637,0.809836,0.806112,0.804511,0.805625,0.631579
2,1.0,0.0,0.0,1.0,1.0,0.428571,0.4,0.00,0.406550,0.627796,0.980064,0.000000,0.562701,0.540193,1.0,0.222222,0.0,0.0,1.0,0.909364,0.388179,0.5,0.363636,0.562092,0.955244,0.407526,0.404476,0.389294,0.389789,0.6240,0.623501,0.602595,0.602107,0.975279,0.975299,0.975318,0.735879,0.3888,0.389289,0.389294,0.389789,0.543307,0.540193,0.540193,0.541833,0.544262,0.907315,0.907268,0.908589,0.842105
3,1.0,0.0,0.0,1.0,0.0,0.285714,0.4,1.00,0.087061,0.141374,0.136364,0.363636,0.562701,0.540193,1.0,0.444444,0.0,0.0,1.0,0.050075,0.116613,1.0,0.272727,0.562092,0.954053,0.087270,0.084732,0.043796,0.000000,0.1368,0.119105,0.043796,0.000000,0.136364,0.118725,0.042994,0.000000,0.1128,0.095923,0.043796,0.000000,0.543307,0.540193,0.540193,0.541833,0.544262,0.050100,0.049624,0.049221,0.105263
4,1.0,0.0,0.0,1.0,0.0,0.857143,0.8,1.00,0.860224,0.857827,0.838915,0.454545,0.565916,0.562701,0.0,0.444444,0.0,0.0,0.0,0.492739,0.873003,0.0,0.363636,0.562092,0.105446,0.859087,0.859313,0.852393,0.749595,0.8592,0.859313,0.852393,0.749595,0.838915,0.837450,0.836783,0.735879,0.8744,0.872102,0.884834,0.749595,0.547244,0.540193,0.540193,0.541833,0.544262,0.492485,0.490727,0.478654,0.263158


In [46]:
mms_test_df = mms_test_df.astype(str)

# 모델 훈련/예측

In [47]:
score_df = pd.DataFrame(index=['thr_전', 'thr_후'])
pred_proba_df = pd.DataFrame()
thr_df = pd.DataFrame()

for random_state in tqdm(range(10, 60, 10)):
  temp_score_list =[]
  # 데이터 셋 나누기
  X_train, X_val, y_train, y_val = train_test_split(mms_X_df, y, random_state=random_state, shuffle=True, stratify=y)
  # 모델 생성
  cat = CatBoostClassifier(random_state=0, task_type='GPU', iterations=1000, early_stopping_rounds=50,
                        cat_features=cat_f, eval_metric='F1', one_hot_max_size=2)
  # 모델 훈련
  cat.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)
  
  # 기본 평가
  val_pred = cat.predict(X_val)
  f1_score_raw = f1_score(y_val, val_pred)
  temp_score_list.append(f1_score_raw)

  # precision, recall, threshold 값 계산
  val_pred_proba = cat.predict_proba(X_val)
  precisions, recalls, thresholds = precision_recall_curve(y_val, val_pred_proba[:,1])
  threshold_boundary = thresholds.shape[0]

  # 최고 F1 값을 갖는 threshold 값 구하기
  pre = precisions[:threshold_boundary]
  re = recalls[:threshold_boundary]
  f1 = 2 * pre * re / (pre + re)
  optimal_idx = np.argmax(f1)
  optimal_threshold = thresholds[optimal_idx]
  thr_df[f'{random_state}'] = [optimal_threshold]

  # 구한 threshold값으로 제한해서 예측
  # 1의 예측값만 불러와도 되므로 [:,1]사용
  val_pred_th =  (val_pred_proba[:,1] >= optimal_threshold)
  val_pred_th = val_pred_th.astype(int)

  # threshold로 제한한 예측 평가
  f1_score_mod = f1_score(y_val, val_pred_th)
  temp_score_list.append(f1_score_mod)
  score_df[f'{random_state}'] = temp_score_list

  print("===============================================")
  print(f"random_state : {random_state}")
  print("threshold 적용 전 f1 score :", f1_score_raw)
  print("threshold 적용 후 f1 score :", f1_score_mod)
  print("=========================================================================================")
  print()

  # test 셋 예측
  pred_proba = cat.predict_proba(mms_test_df)
  pred_proba_df[f'{random_state}'] = pred_proba[:,1]

# about 11m

  0%|          | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.043012
0:	learn: 0.6178795	test: 0.6270133	best: 0.6270133 (0)	total: 348ms	remaining: 5m 47s
100:	learn: 0.6488142	test: 0.6629757	best: 0.6631683 (98)	total: 32.4s	remaining: 4m 48s
200:	learn: 0.6589275	test: 0.6763521	best: 0.6763934 (198)	total: 1m 5s	remaining: 4m 18s
300:	learn: 0.6637678	test: 0.6791240	best: 0.6795223 (290)	total: 1m 37s	remaining: 3m 46s
400:	learn: 0.6670909	test: 0.6805284	best: 0.6805284 (400)	total: 2m 9s	remaining: 3m 13s
bestTest = 0.6808433434
bestIteration = 410
Shrink model to first 411 iterations.
random_state : 10
threshold 적용 전 f1 score : 0.6808433434379324
threshold 적용 후 f1 score : 0.7101440998991111



 20%|██        | 1/5 [03:02<12:11, 182.75s/it]

Learning rate set to 0.043012
0:	learn: 0.6183910	test: 0.6191865	best: 0.6191865 (0)	total: 349ms	remaining: 5m 49s
100:	learn: 0.6485855	test: 0.6609577	best: 0.6609577 (100)	total: 32.1s	remaining: 4m 45s
200:	learn: 0.6598716	test: 0.6771290	best: 0.6772523 (193)	total: 1m 4s	remaining: 4m 15s
300:	learn: 0.6647198	test: 0.6807145	best: 0.6808479 (298)	total: 1m 36s	remaining: 3m 45s
400:	learn: 0.6680912	test: 0.6821081	best: 0.6822367 (394)	total: 2m 9s	remaining: 3m 13s
bestTest = 0.6822366978
bestIteration = 394
Shrink model to first 395 iterations.
random_state : 20
threshold 적용 전 f1 score : 0.6822366977613351
threshold 적용 후 f1 score : 0.7097142710445922



 40%|████      | 2/5 [06:00<08:59, 179.89s/it]

...

 60%|██████    | 3/5 [08:56<05:56, 178.25s/it]

Learning rate set to 0.043012
0:	learn: 0.6014546	test: 0.6019652	best: 0.6019652 (0)	total: 348ms	remaining: 5m 48s
100:	learn: 0.6491838	test: 0.6647142	best: 0.6648192 (99)	total: 32s	remaining: 4m 44s
200:	learn: 0.6578757	test: 0.6773255	best: 0.6773255 (200)	total: 1m 4s	remaining: 4m 14s
300:	learn: 0.6627565	test: 0.6808895	best: 0.6812633 (289)	total: 1m 36s	remaining: 3m 44s
bestTest = 0.6812633211
bestIteration = 289
Shrink model to first 290 iterations.
random_state : 40
threshold 적용 전 f1 score : 0.6812633210618098
threshold 적용 후 f1 score : 0.7067351865815871



 80%|████████  | 4/5 [11:19<02:44, 164.13s/it]

Learning rate set to 0.043012
0:	learn: 0.6203906	test: 0.6263513	best: 0.6263513 (0)	total: 353ms	remaining: 5m 52s
100:	learn: 0.6493087	test: 0.6630866	best: 0.6632034 (99)	total: 32.1s	remaining: 4m 45s
200:	learn: 0.6588583	test: 0.6773647	best: 0.6774653 (199)	total: 1m 4s	remaining: 4m 14s
300:	learn: 0.6633106	test: 0.6789820	best: 0.6791208 (295)	total: 1m 36s	remaining: 3m 43s
400:	learn: 0.6671982	test: 0.6797972	best: 0.6804773 (356)	total: 2m 9s	remaining: 3m 14s
bestTest = 0.6804773428
bestIteration = 356
Shrink model to first 357 iterations.
random_state : 50
threshold 적용 전 f1 score : 0.6804773428017882
threshold 적용 후 f1 score : 0.7096340287295061



100%|██████████| 5/5 [14:04<00:00, 168.98s/it]


## 결과 종합

In [48]:
score_df

,10,20,30,40,50
thr_전,0.680843,0.682237,0.681525,0.681263,0.680477
thr_후,0.710144,0.709714,0.709201,0.706735,0.709634


### 오류난 특성 제거

In [ ]:
# del_col = ['40']
# pred_proba_df = pred_proba_df.drop(del_col, axis=1)
# thr_df = thr_df.drop(del_col, axis=1)
# score_df = score_df.drop(del_col, axis=1)

In [49]:
pred_proba_df['mean'] = pred_proba_df.mean(axis=1)
thr_df['mean'] = thr_df.mean(axis=1)
score_df['mean'] = score_df.mean(axis=1)

### threshold 값

In [50]:
thr_df

,10,20,30,40,50,mean
0,0.381403,0.391302,0.376105,0.39377,0.385471,0.38561


## 자체평가

In [51]:
score_df

,10,20,30,40,50,mean
thr_전,0.680843,0.682237,0.681525,0.681263,0.680477,0.681269
thr_후,0.710144,0.709714,0.709201,0.706735,0.709634,0.709086


## test 예측

In [52]:
pred = pred_proba_df['mean'].tolist() >= thr_df['mean'].values
pred = pred.astype(int)
pred

array([1, 1, 1, ..., 1, 1, 1])

In [53]:
np.unique(pred, return_counts=True)

(array([0, 1]), array([ 1130, 45274]))

# 제출파일 불러오기

In [54]:
# sub = pd.read_csv('../data/sample_submission.csv')
sub = pd.read_csv('/content/drive/MyDrive/GitHub/Dacon/TP2(잡케어)/data/sample_submission.csv')
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      46404 non-null  int64
 1   target  46404 non-null  int64
dtypes: int64(2)
memory usage: 725.2 KB


# 제출파일 생성

In [55]:
sub['target'] = pred
sub['target'].value_counts()

1    45274
0     1130
Name: target, dtype: int64

In [56]:
file_ver = '6.2'
# sub.to_csv(f'sub_ver{file_ver}.csv', index=False)
sub.to_csv(f'/content/drive/MyDrive/GitHub/Dacon/TP2(잡케어)/03_모델선정및개선/sub_ver{file_ver}.csv', index=False)

# 코드 마지막